# Reconhecimento de flores e árvores

## Autora
* Isabella Carneiro Godinho de Morais Sá
* Matrícula: 18/0019066
* Github: isabellacgmsa

## Objetivo
O propósito do presente artigo é demonstrar o conhecimento adquirido através da primeira lição e os passos que foram realizados para treinar a identificação de flores através de imagens.

## Dependências
Para alcançar o objetivo, o primeiro passo tomado foi a instalação das dependências necessárias com os seguintes comandos:

```pip install -U duckduckgo_search```

```pip install fastai"```

Lembrando que, antes de iniciar o processo é necessário fazer a instalação do PyTorch.

## Baixando imagens
Para fazer a instalação das imagens foi utilizado o ddg_images, como mostra o código a seguir:  




```
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term: str, max_images=30):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')
    
```

O primeiro teste será feito baixando apenas uma imagem de flor:

```
urls = search_images('flower', max_images=1)
urls[2]
```

O download será realizado utilizando a biblioteca fastdownload:

```
from fastdownload import download_url
dest = 'oneflower.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)
```

![texto](https://s3.static.brasilescola.uol.com.br/be/2020/12/girassol.jpg)

Ao constatar que está funcionando todo o processo é possível fazer o download de dados mais numerosos:
```
searches = 'tree', 'flower'
path = Path('tree_flower')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} tree'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} flower'))
    sleep(10)
    resize_images(path/o, max_size=400, dest= path/o)
```

Foi utilizado o sleep para que não haja problemas no tráfego de informações.

Essas imagens serão salvas em uma pasta cujo caminho está definido no path.


## Treinamento
### Passo 1
Antes de iniciar o treinamento do modelo de fato é necessário garantir que não haja imagens com erro para evitar futuros problemas:

```
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)
```
### Passo 2

Para treinar modelos de aprendizado de máquina com conjuntos de dados volumosos, é necessário dividir os dados em lotes menores para evitar sobrecarregar a memória RAM do computador. O Dataloader é uma classe que oferece essa funcionalidade de forma eficiente. Ele carrega e processa os dados em lotes menores, tornando o processo de treinamento escalável.

O fastai é um framework de aprendizado de máquina que permite o uso do datablock, uma ferramenta que facilita a personalização do processamento e organização dos dados em lotes para treinamento do modelo. Com o datablock, é possível definir etapas como leitura, pré-processamento e organização dos dados de forma mais simples. Isso permite que os usuários criem fluxos de trabalho personalizados e experimentem diferentes abordagens para melhorar a precisão do modelo.

```
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)
```
![texto](https://i.ibb.co/T4XzYk1/flores.png)

Foi especificado que há o ImageBlock, que é uma classe do fastai que permite a criação de blocos para processar as imagens no modelo, e o CategoryBlock que cria blocos para processar variáveis categóricas, nesse caso árvore ou flor. Nesse trecho também especifica o tamanho que as imagens terão, já que precisa ser o mesmo para todas.

### Passo 3
Após criar o Datablock, o modelo é treinado com a ajuda do objeto "Learner". Ele é configurado com três argumentos: o primeiro é um DataLoader (dls) que contém o conjunto de dados de treinamento do modelo; o segundo é a arquitetura ResNet18 que será usada como base para o modelo; e o terceiro é a métrica usada para avaliar a performance do modelo, que neste caso é a taxa de erro.

Para realizar o ajuste fino do modelo, é utilizado o método "fine_tune()". Esse método realiza um treinamento adicional do modelo usando os pesos pré-treinados da ResNet18 como ponto de partida.
```
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(8)
```

### Passo 4
Por fim, foi usada uma imagem de uma flor para testar. O retorno foi:

```
É uma flor.
Probabilidade de ser uma flor: 96.37%
```

## Conclusão
A fastai torna mais simples e eficiente o processo de treinar um modelo para reconhecimento através de imagens. O principal problema foi uma dificuldade pessoal sobre a linguagem utilizada, visto que não tinha conhecimento prévio, porém utilizei da semana inicial que não havia recebido ainda as instruções de como funcionaria a disciplina para focar em entender Python.
Após superar esse contratempo gostei muito do material disponibilizado e com o mesmo foi possível compreender e ter um ótimo contato inicial com aprendizado de máquina.